In [ ]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
from AnsatzLacs import *
from PurificationUtil import *
from qiskit_algorithms.optimizers import SPSA

In [ ]:
# implementation of the slack variable method
# quantum circuit settings
# these values will be used as global variables later in different functions
sys_size = 2 # size of the input state (2 means 2 qubit)
ref_size = 2 # size of the reference state (for purification)

swap_test_unitary = get_swap_unitary(sys_size)

In [ ]:
#Problem setup
pauli = ['ZZ','XI','IX','YI','IZ']
H_paulis = ['ZZ','XI','IX']
expectation_unitary_dict = {pauli_string: get_expectation_unitary(pauli_string) for pauli_string in pauli}

a1 = 0.2
a2 = 0.1
b1 = a1 # for the dual
b2 = a2 # for the dual

In [ ]:
#CVXpy solutions
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1,0], [0 ,-1]])
Y = np.array([[0, -1j], [1j, 0]])
H = np.kron(Z,Z) + np.kron(np.eye(2), X) + np.kron(X, np.eye(2))
A_2 = np.kron(np.eye(2),Z)
A_1 = np.kron(Y,np.eye(2))
rho = cp.Variable( np.eye(4).shape, hermitian=True)
constraints = [rho >> 0, cp.real(cp.trace(A_1 @ rho)) >= a1,cp.real(cp.trace(A_2 @ rho)) >= a2, cp.trace(rho) ==1]
prob = cp.Problem(cp.Minimize(cp.real(cp.trace(H @ rho))),
                  constraints)
# Solve SDP
prob.solve(verbose=False)
expected = prob.value
print("Aprroximated value:", prob.value)

In [ ]:
def get_trace(ansatz1, ansatz2, shots):
    return get_trace_helper(swap_test_unitary, ansatz1, ansatz2, shots)

def get_exp(pauli_string, ansatz, shots):
    return get_exp_helper(expectation_unitary_dict, pauli_string, ansatz, shots)

In [ ]:
# precompute terms
# Tr[H^2]=12, Tr[A1^2]=Tr[A2^2]=4
TrH2 = 12
TrA_12 = 4
TrA_22 = 4

# Primal Optimization

In [ ]:
def primal_cost_function_full(params):
    # update parameters
    lmbda = params[-2] ** 2
    mu = params[-1] ** 2
     
    params = params[:primal_num_params]
    omega.set_params(params)
  
    # calculate new cost
    obj = 0
    for i in range(len(H_paulis)):
        obj += get_exp(H_paulis[i], omega, num_shots)

    est2 = get_exp(pauli[3], omega, num_shots)
    est3 = get_exp(pauli[4], omega, num_shots)
    penalty = (lmbda - est2 + a1)**2 + (mu - est3 + a2)**2
    cost = obj + c * penalty
    return cost, obj, penalty

def primal_cost_function(params):
    cost, _, _ = primal_cost_function_full(params)
    return cost

In [ ]:
print_every = 100
def primal_callback_spsa(nfev, x, fx, stepsize, accepted=False):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    global primal_loss
    global primal_penals
    global primal_ojectives
    
    fx, obj, penal = primal_cost_function_full(x)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)
        
    if(nfev % print_every == 0):
        print(f"Iteration: {nfev//3}\tCost: {round(fx,4)}\tObj: {round(obj, 4)}\tPenalty: {round(penal, 4)}\tc: {c}")

In [ ]:
num_shots = 1e12
max_iter = 20 * 1000

lr_max = 1e-1
lr_min = 1e-3
dec_interval = 1000
train_lr = get_lr(lr_max, lr_min, dec_interval, max_iter) # halve the lr every 1000 iterations

pertb = 1e-4
trust = True

primal_num_layers = 2
primal_num_params = get_parameter_number("general", sys_size, ref_size, primal_num_layers)

In [ ]:
num_runs = 3
record = 0 # record the number of successful trails

costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

for i in range(num_runs):
    np.random.seed(i)
    lmbda = 0.1
    mu = 0.5
    c = 100

    lr = lr_max
    
    primal_loss = []
    primal_penals = []
    primal_objectives = []

    print(f"Exact value: {expected}")
    omega = create_ansatz("general", sys_size, ref_size, primal_num_layers)
    init_params = [*omega.params, np.sqrt(lmbda), np.sqrt(mu)]
    print(f"Iteration {i}, Initial cost : {primal_cost_function(init_params)}")
    
    autospsa = SPSA(maxiter=max_iter, learning_rate=train_lr,
                        perturbation=pertb, callback=primal_callback_spsa, trust_region=trust)
    obj = autospsa.minimize(fun=primal_cost_function, x0=init_params)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)

    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"CH_Purification_primal_data"
        problem_name = "CH"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}")


# Dual Optimization

In [ ]:
N = 2 ** sys_size
def dual_cost_function_full(params):
    # update parameters
    y1 = params[0] ** 2
    y2 = params[1] ** 2
    nu = params[2] ** 2
    mu = params[3]      # mu only needs to be real number
    
    params = params[4:]
    omega.set_params(params)

    # calculate objective function
    # Tr[Hω] 
    t1 = get_exp(pauli[0], omega, num_shots) 
    t2 = get_exp(pauli[1], omega, num_shots)
    t3 = get_exp(pauli[2], omega, num_shots)
    tr_H_omega = t1 + t2 + t3 

    # y_0Tr[A_0ω] + y_1Tr[A_1ω]
    est2 = get_exp(pauli[3], omega, num_shots)
    est3 = get_exp(pauli[4], omega, num_shots)
    tr_A_omega = y1 * est2 + y2 * est3

    omega_sqr = get_trace(omega, omega, num_shots)
    
    # corresponds to the three lines of equation G101
    # certain terms are omitted for this particular example
    penalty1 = TrH2 + N * (y1 ** 2 + y2 ** 2 + mu ** 2)
    penalty2 = (nu ** 2) * omega_sqr - 2 * nu * tr_H_omega
    penalty3 = 2 * nu * tr_A_omega + 2 * mu * nu

    obj = b1 * y1 + b2 * y2 + mu
    penalty = penalty1 + penalty2 + penalty3
    cost = obj - c * penalty
    return cost, obj, penalty

def dual_cost_function(params):
    cost, _, _ = dual_cost_function_full(params)
    return -cost

In [ ]:
print_every = 100
def dual_callback_spsa(nfev, x, fx, stepsize, accepted=False):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    global dual_loss
    global dual_penals
    global dual_ojectives
    
    fx, obj, penal = dual_cost_function_full(x)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objectives.append(obj)
        
    if(nfev % print_every == 0):
        print(f"Iteration: {nfev//3}\tCost: {round(fx,4)}\tObj: {round(obj, 4)}\tPenalty: {round(penal, 4)}\tc: {c}")

In [ ]:
num_shots = 1e12
max_iter = 80 * 1000

lr_max = 1e-1
lr_min = 1e-3
dec_interval = 1000
train_lr = get_lr(lr_max, lr_min, dec_interval, max_iter) # halve the lr every 1000 iterations
perturbation = 1e-4

trust = True
dual_num_layers = 2

In [ ]:
num_runs = 3
record = 0 # record the number of successful trails

costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []

for i in range(num_runs):
    np.random.seed(i)
    y1 = 0.001
    y2 = 0.001
    mu = -0.005
    nu = 0.001
    
    c = 100

    dual_loss = []
    dual_penals = []
    dual_objectives = []

    print(f"Exact value: {expected}")
    omega = create_ansatz("general", sys_size, ref_size, dual_num_layers)
    init_params = [np.sqrt(y1), np.sqrt(y2), np.sqrt(nu), mu, *omega.params]
    print(f"Iteration {i}, Initial cost : {-dual_cost_function(init_params)}")
    
    autospsa = SPSA(maxiter=max_iter, learning_rate=train_lr,
                        perturbation=pertb, callback=dual_callback_spsa, trust_region=trust)
    obj = autospsa.minimize(fun=dual_cost_function, x0=init_params)
    
    costs_dual_outer.append(dual_loss)
    objs_dual_outer.append(dual_objectives)
    penalties_dual_outer.append(dual_penals)

    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"CH_Purification_dual_data"
        problem_name = "CH"
        filename = f"Init_{i}"
        save_data_to_file(dual_loss, dual_penals, dual_objectives, problem_name, dir_name, filename)
        record += 1
print(f"Number of successful runs: {record}")


# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 100
window_size_dual = 100

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
x1 = len(costs_primal_median)
x2 = len(costs_dual_median)
x3 = max(x1, x2)
expected_line = np.zeros(x3) + expected
plt.plot(np.arange(x1), costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(np.arange(x2), costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(x3), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(np.arange(x1), costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(np.arange(x2), costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
# plt.ylim(0, .5)
plt.xlim(0, max_iter)

# Display the graph
plt.show()